In [1]:
from bbSearchRaw import SearchProblem, search, thecost

Loading bbSearch Version 2.1 (at 21:37, Tue 01 Apr)
Last module source code edit 9am Thursday 24th Feb 2022


In [2]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength

    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [3]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )


In [4]:
from copy import deepcopy

class RobotWorker( SearchProblem ):

    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions( self, state ):

        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()

        actions = []
        # Can put down any carried item
        for i in state.robot.carried_items:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough修改
        for i in state.room_contents[robot_location]:
            if ITEM_WEIGHT[i]==0:
                actions.append( ("pick up", i))
            elif any(i in goal_items for goal_items in self.goal_item_locations.values()):
                if strength >= weight_carried + ITEM_WEIGHT[i]:
                    actions.append(("pick up", i))

        # If there is an unlocked door between robot location and
        # another location can move to that location
#         print(state.doors)
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )
            elif door.locked==True and door.doorkey in state.robot.carried_items and robot_location in door.goes_between:
#                 print(door)
#                 print(robot_location)
#                 print(door.doorkey)
#                 print(door.other_loc[robot_location])
                actions.append( ("move to", door.other_loc[robot_location]) )
        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True

    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)

## small

In [27]:
ROOM_CONTENTS = {
    'workshop'     : {'key'},
    'store room'   : {'bucket', 'suitcase'},
    'tool cupboard' : {'hammer', 'anvil', 'saw', 'screwdriver'},
}

ITEM_WEIGHT = {
    'key' : 0,
    'bucket' : 2,
    'suitcase' : 4,
    'screwdriver' : 1,
    'hammer' : 5,
    'anvil' : 12,
    'saw' : 2,
}

DOORS = [
    Door('workshop', 'store room' ),
    Door( 'store room', 'tool cupboard', doorkey='key', locked= False )
]


In [28]:
rob = Robot('store room', [], 15 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {"store room":{"hammer", "screwdriver", "anvil"}}

RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )

## middle

In [42]:
ROOM_CONTENTS = {
    'workshop'     : {'key1'},
    'store room'   : {'bucket', 'suitcase','key2'},
    'tool cupboard' : {'hammer', 'anvil', 'saw', 'screwdriver'},
    'warehouse': {'box', 'crate', 'container'}
}

ITEM_WEIGHT = {
    'key1' : 0,
    'bucket' : 2,
    'suitcase' : 4,
    'screwdriver' : 1,
    'hammer' : 5,
    'anvil' : 12,
    'saw' : 2,
    'box':3,
    'crate':8,
    'container':10,
    'key2':0
}

DOORS = [
    Door('workshop', 'store room' ),
    Door( 'store room', 'tool cupboard', doorkey='key1', locked=True),
    Door( 'workshop', 'warehouse', doorkey='key2', locked=True ),
    Door('warehouse','tool cupboard')
]

In [43]:
rob = Robot('store room', [], 15 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {
    "workshop":{"box"},
    "store room":{"hammer", "screwdriver", "anvil"},
    "warehouse" :{"bucket"}
}

RW_PROBLEM_2 = RobotWorker( state, goal_item_locations )

## hard

In [5]:
# ROOM_CONTENTS = {
#     'workshop'     : {'key1'},
#     'store room'   : {'bucket', 'suitcase','key2'},
#     'tool cupboard' : {'anvil', 'saw','thing3'},
#     'warehouse': {'box', 'crate', 'container'},
#     'room1': {'thing1','thing2'},
#     'room3':{'key3'}
# }

# ITEM_WEIGHT = {
#     'key1' : 0,
#     'key2':0,
#     'key3':0,
#     'bucket' : 2,
#     'suitcase' : 4,
#     'screwdriver' : 1,
#     'hammer' : 5,
#     'anvil' : 12,
#     'saw' : 2,
#     'box':3,
#     'crate':8,
#     'container':10,
#     'thing1': 6,
#     'thing2': 4,
#     'thing3': 3
# }

# DOORS = [
#     Door('workshop', 'store room' ),
#     Door('room3', 'tool cupboard' ),
#     Door('warehouse', 'room1' ),
#     Door( 'store room', 'room3', doorkey='key1', locked=True),
#     Door( 'workshop', 'warehouse', doorkey='key2', locked=True ),
#     Door( 'room1', 'tool cupboard', doorkey='key3', locked=True )
# ]

In [52]:
## 有锁无key
ROOM_CONTENTS = {
    'workshop'     : {'key1'},
    'store room'   : {'bucket', 'suitcase'},
    'tool cupboard' : {'anvil', 'saw'},
    'warehouse': {'box', 'crate', 'container','key4'},
    'room1': {'thing1','thing2'},
    'room2':{'thing3'},
    'room3':{'key2'}
}

ITEM_WEIGHT = {
    'key1' : 0,
    'key2':0,
    'key4':0,
    'bucket' : 2,
    'suitcase' : 4,
    'screwdriver' : 1,
    'hammer' : 5,
    'anvil' : 12,
    'saw' : 2,
    'box':3,
    'crate':8,
    'container':10,
    'thing1': 6,
    'thing2': 7,
    'thing3': 3
}

DOORS = [
    Door('workshop', 'store room' ),
    Door('room3', 'room2' ),
    Door('warehouse', 'room1' ),
    Door( 'store room', 'room3', doorkey='key1', locked=True),
    Door( 'workshop', 'warehouse', doorkey='key2', locked=True ),
    Door( 'room1', 'tool cupboard', doorkey='key3', locked=True ),
    Door( 'room2', 'tool cupboard', doorkey='key4', locked=True )
]

In [53]:
# rob = Robot('store room', [], 15 )

# state = State(rob, DOORS, ROOM_CONTENTS)

# goal_item_locations =  {
#     "store room":{"thing1", "box"},
#     "room3" :{"saw"},
#     'warehouse':{'bucket','suitcase'},
#     'room1':{'anvil'}
# }

# RW_PROBLEM_3 = RobotWorker( state, goal_item_locations )
rob = Robot('store room', [], 15 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {
    "store room":{"thing1", "thing2", "box"},
    "room3" :{"saw"},
    'warehouse':{'bucket','suitcase'},
    'room1':{'anvil'},
    "tool cupboard":{'thing3'}
}

RW_PROBLEM_3 = RobotWorker( state, goal_item_locations )

## 没有启发式

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=True, randomise=False)

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=False, randomise=False)

In [ ]:
search( RW_PROBLEM_1, 'DF/LIFO', 10000000, loop_check=True, randomise=False)

In [ ]:
search( RW_PROBLEM_1, 'DF/LIFO', 10000000, loop_check=False, randomise=False)

In [ ]:
search( RW_PROBLEM_1, 'DF/LIFO', 10000000, loop_check=True, randomise=True)  

In [ ]:
search( RW_PROBLEM_1, 'DF/LIFO', 10000000, loop_check=False, randomise=True)

In [60]:
# search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=True, randomise=False)
import threading
def target():
        result[0] = search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=True, randomise=False)
thread = threading.Thread(target=target)
thread.start()
thread.join(timeout=300) 
if thread.is_alive():  
    print("The search times out")
else: 
    result[0]

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
.................................................................................................... (200000)
.................................................................................................... (300000)
...................................................................................................The search times out
. (400000)
.................

In [ ]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=False, randomise=False)

In [ ]:
search( RW_PROBLEM_2, 'DF/LIFO', 10000000, loop_check=True, randomise=False)

In [ ]:
search( RW_PROBLEM_2, 'DF/LIFO', 10000000, loop_check=False, randomise=False)

In [ ]:
search( RW_PROBLEM_2, 'DF/LIFO', 10000000, loop_check=True, randomise=True)

In [ ]:
search( RW_PROBLEM_2, 'DF/LIFO', 10000000, loop_check=False, randomise=True)

In [ ]:
# search( RW_PROBLEM_3, 'BF/FIFO', 10000000, loop_check=True, randomise=False)
import threading
def target():
        result[0] = search( RW_PROBLEM_3, 'BF/FIFO', 10000000, loop_check=True, randomise=False)
thread = threading.Thread(target=target)
thread.start()
thread.join(timeout=300) 
if thread.is_alive():  
    print("The search times out")
else: 
    result[0]

In [ ]:
import threading
def target():
        result[0] = search( RW_PROBLEM_3, 'DF/LIFO', 10000000, loop_check=True, randomise=False)
thread = threading.Thread(target=target)
thread.start()
thread.join(timeout=300) 
if thread.is_alive():  
    print("The search times out")
else: 
    result[0]

In [ ]:
search( RW_PROBLEM_3, 'DF/LIFO', 10000000, loop_check=True, randomise=True)

## 启发式

In [31]:
def misplace (state,goal_position):
    total_distance=0
    for room, contents in goal_position.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    total_distance +=1 
    return total_distance*4

In [32]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=True, heuristic=misplace)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total nodes generated          =     

'GOAL_STATE_FOUND'

In [33]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=False, heuristic=misplace)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total nodes generated          =     

'GOAL_STATE_FOUND'

In [34]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=True, heuristic=misplace, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
Cost of reaching goal: 12
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total no

'GOAL_STATE_FOUND'

In [35]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=False, heuristic=misplace, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
Cost of reaching goal: 12
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total no

'GOAL_STATE_FOUND'

In [44]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=True, heuristic=misplace)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put do

'GOAL_STATE_FOUND'

In [45]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=False, heuristic=misplace)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put do

'GOAL_STATE_FOUND'

In [46]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=True, heuristic=misplace, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
Cost of reaching goal: 22
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to

'GOAL_STATE_FOUND'

In [47]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=False, heuristic=misplace, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...................
:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
Cost of reaching goal: 22
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'ham

'GOAL_STATE_FOUND'

In [16]:
search( RW_PROBLEM_3, 'BF/FIFO', 10000000, loop_check=True, heuristic=misplace)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
....
:-)) *SUCCESS* ((-:

Path length = 56
Goal state is:
Robot location: room1
Robot carrying: ['key1', 'key2', 'key4']
Room contents: {'workshop': set(), 'store room': {'box', 'thing2', 'thing1'}, 'tool cupboard': {'thing3'}, 'warehouse': {'suitcase', 'container', 'bucket', 'crate'}, 'room1': {'anvil'}, 'room2': set(), 'room3': {'saw'}}
The action path to the solution is:
    ('pick up', 'suitcase')
    ('move to', 'workshop')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'room3')
    ('pick up', 'key2')
    ('move to', 'store room')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'suitcase')
    ('pick up', 'box')
    ('move 

'GOAL_STATE_FOUND'

In [54]:
search( RW_PROBLEM_3, 'BF/FIFO', 10000000, loop_check=False, heuristic=misplace)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=misplace
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
........................................................

KeyboardInterrupt: 

In [17]:
search( RW_PROBLEM_3, 'BF/FIFO', 1000000, loop_check=True, heuristic=misplace, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=misplace
Max search nodes: 1000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
..............................................................................
:-)) *SUCCESS* ((-:

Path length = 41
Goal state is:
Robot location: store room
Robot carrying: ['key1', 'key2', 'key4']
Room contents: {'workshop': set(), 'store room': {'box', 'thing2', 'thing1'}, 'tool cupboard': {'thing3'}, 'warehouse': {'crate', 'container', 'bucket', 'suitcase'}, 'room1': {'anvil'}, 'room2': set(), 'room3': {'saw'}}
Cost of reaching goal: 41
The action path to the solution is:
    ('pick up', 'suitcase')
    ('pick up', 'bucket')
    ('move to', 'workshop')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'room3')
    ('pick up', 'key2')
    ('move to', '

'GOAL_STATE_FOUND'

In [55]:
search( RW_PROBLEM_3, 'BF/FIFO', 1000000, loop_check=False, heuristic=misplace, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=misplace
Max search nodes: 1000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
....................

KeyboardInterrupt: 

In [37]:
def carry_item (state, goal_position):
    total_distance = 0
    robot_location = state.robot.location
    carried_items = state.robot.carried_items

    for room, contents in goal_position.items():
        for item in contents:
            if item not in carried_items and item not in state.room_contents[room] :
                total_distance += 1
            # If the target item is on the robot and the robot is not in the target room, 
            # the robot may need to go back and put the item
            elif item in carried_items:
                if robot_location != room:
                    total_distance += 1 
    return total_distance*4

In [38]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=True, heuristic=carry_item)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total nodes generated          =   

'GOAL_STATE_FOUND'

In [39]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=False, heuristic=carry_item)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total nodes generated          =   

'GOAL_STATE_FOUND'

In [40]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=True, heuristic=carry_item,cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
Cost of reaching goal: 12
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total 

'GOAL_STATE_FOUND'

In [41]:
search( RW_PROBLEM_1, 'BF/FIFO', 10000000, loop_check=False, heuristic=carry_item, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 12
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'key'}, 'store room': {'suitcase', 'bucket', 'screwdriver', 'anvil', 'hammer'}, 'tool cupboard': {'saw'}}
Cost of reaching goal: 12
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put down', 'hammer')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'screwdriver')


SEARCH SPACE STATS:
Total 

'GOAL_STATE_FOUND'

In [48]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=True, heuristic=carry_item)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put 

'GOAL_STATE_FOUND'

In [49]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=False, heuristic=carry_item)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move to', 'store room')
    ('put 

'GOAL_STATE_FOUND'

In [50]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=True, heuristic=carry_item, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
Cost of reaching goal: 22
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'hammer')
    ('move 

'GOAL_STATE_FOUND'

In [51]:
search( RW_PROBLEM_2, 'BF/FIFO', 10000000, loop_check=False, heuristic=carry_item, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................
:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'key1']
Room contents: {'workshop': {'box'}, 'store room': {'hammer', 'suitcase', 'anvil', 'screwdriver'}, 'tool cupboard': {'saw'}, 'warehouse': {'crate', 'container', 'bucket'}}
Cost of reaching goal: 22
The action path to the solution is:
    ('pick up', 'bucket')
    ('pick up', 'key2')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'bucket')
    ('pick up', 'box')
    ('move to', 'workshop')
    ('put down', 'box')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'ham

'GOAL_STATE_FOUND'

In [56]:
search( RW_PROBLEM_3, 'BF/FIFO', 10000000, loop_check=True, heuristic=carry_item)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.......
:-)) *SUCCESS* ((-:

Path length = 56
Goal state is:
Robot location: room1
Robot carrying: ['key1', 'key2', 'key4']
Room contents: {'workshop': set(), 'store room': {'box', 'thing2', 'thing1'}, 'tool cupboard': {'thing3'}, 'warehouse': {'suitcase', 'container', 'bucket', 'crate'}, 'room1': {'anvil'}, 'room2': set(), 'room3': {'saw'}}
The action path to the solution is:
    ('pick up', 'suitcase')
    ('move to', 'workshop')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'room3')
    ('pick up', 'key2')
    ('move to', 'store room')
    ('move to', 'workshop')
    ('move to', 'warehouse')
    ('put down', 'suitcase')
    ('pick up', 'box')
    ('

'GOAL_STATE_FOUND'

In [57]:
search( RW_PROBLEM_3, 'BF/FIFO', 1000000, loop_check=True, heuristic=carry_item, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=carry_item
Max search nodes: 1000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.............................................................................................
:-)) *SUCCESS* ((-:

Path length = 41
Goal state is:
Robot location: store room
Robot carrying: ['key1', 'key2', 'key4']
Room contents: {'workshop': set(), 'store room': {'box', 'thing2', 'thing1'}, 'tool cupboard': {'thing3'}, 'warehouse': {'crate', 'container', 'bucket', 'suitcase'}, 'room1': {'anvil'}, 'room2': set(), 'room3': {'saw'}}
Cost of reaching goal: 41
The action path to the solution is:
    ('pick up', 'suitcase')
    ('pick up', 'bucket')
    ('move to', 'workshop')
    ('pick up', 'key1')
    ('move to', 'store room')
    ('move to', 'room3')
    ('pick up', 'key2')


'GOAL_STATE_FOUND'

In [58]:
search( RW_PROBLEM_3, 'BF/FIFO', 10000000, loop_check=False, heuristic=carry_item)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=carry_item
Max search nodes: 10000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
........................

KeyboardInterrupt: 

In [59]:
search( RW_PROBLEM_3, 'BF/FIFO', 1000000, loop_check=False, heuristic=carry_item, cost=thecost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=thecost, heuristic=carry_item
Max search nodes: 1000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
..............................................

KeyboardInterrupt: 